In [ ]:
# based on https://til.simonwillison.net/llms/python-react-pattern

In [1]:
import openai
import re
import httpx
import os
# from dotenv import load_dotenv

# _ = load_dotenv()
# from openai import OpenAI
from openai import OpenAI
client = OpenAI(base_url="http://10.2.125.37:1234/v1", api_key="lm-studio")

In [2]:
# chat_completion = client.chat.completions.create(
#     model="gpt-3.5-turbo",
#     messages=[{"role": "user", "content": "Hello world"}]
# )

chat_completion = client.chat.completions.create(
  model="lmstudio-community/Meta-Llama-3.1-8B-Instruct-GGUF",
  messages=[
    # {"role": "system", "content": "Always answer in rhymes."},
    {"role": "user", "content": "Hello world"}
  ],
  temperature=0.7,
)

print(chat_completion.choices[0].message)

ChatCompletionMessage(content='It seems like you\'re trying to start with a classic "hello world" program, which is often used as a simple example in programming.\n\nHere\'s a basic Python code that prints "Hello World":\n\n```python\nprint("Hello World")\n```\n\nIf you want to create an executable file for this code (e.g., on Windows or macOS), here are the steps:\n\n1.  Open your preferred text editor or IDE.\n2.  Type in `print("Hello World")` and save it with a `.py` extension, e.g., `hello_world.py`.\n3.  On most operating systems, you can run this code by opening a terminal or command prompt, navigating to the directory where you saved the file, and typing `python hello_world.py`.\n\nHowever, if you\'re looking for something more interactive or want to create a graphical interface, there are many other libraries (e.g., Tkinter, PyQt) that allow you to do so.\n\nLet me know what you\'d like to achieve, and I can provide more tailored assistance!', role='assistant', function_call=N

In [3]:
chat_completion.choices[0].message.content

'It seems like you\'re trying to start with a classic "hello world" program, which is often used as a simple example in programming.\n\nHere\'s a basic Python code that prints "Hello World":\n\n```python\nprint("Hello World")\n```\n\nIf you want to create an executable file for this code (e.g., on Windows or macOS), here are the steps:\n\n1.  Open your preferred text editor or IDE.\n2.  Type in `print("Hello World")` and save it with a `.py` extension, e.g., `hello_world.py`.\n3.  On most operating systems, you can run this code by opening a terminal or command prompt, navigating to the directory where you saved the file, and typing `python hello_world.py`.\n\nHowever, if you\'re looking for something more interactive or want to create a graphical interface, there are many other libraries (e.g., Tkinter, PyQt) that allow you to do so.\n\nLet me know what you\'d like to achieve, and I can provide more tailored assistance!'

In [4]:
class Agent:
    def __init__(self, system=""):
        self.system = system
        self.messages = []
        if self.system:
            self.messages.append({"role": "system", "content": system})

    def __call__(self, message):
        self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result

    def execute(self):
        completion = client.chat.completions.create(
                        model="lmstudio-community/Meta-Llama-3.1-8B-Instruct-GGUF", 
                        temperature=0,
                        messages=self.messages)
        return completion.choices[0].message.content
    

In [5]:
prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

average_dog_weight:
e.g. average_dog_weight: Collie
returns average weight of a dog when given the breed

Example session:

Question: How much does a Bulldog weigh?
Thought: I should look the dogs weight using average_dog_weight
Action: average_dog_weight: Bulldog
PAUSE

You will be called again with this:

Observation: A Bulldog weights 51 lbs

You then output:

Answer: A bulldog weights 51 lbs
""".strip()

In [52]:
def calculate(what):
    return eval(what)
# def calculate(what):
#     if what.startswith("average_dog_weight:"):
#         dog_name = what.split(":")[1].strip()
#         return average_dog_weight(dog_name)
#     else:
#         return eval(what)

def average_dog_weight(name):
    if name in "Scottish Terrier": 
        return("Scottish Terriers average 20 lbs")
    elif name in "Border Collie":
        return("a Border Collies average weight is 37 lbs")
    elif name in "Toy Poodle":
        return("a toy poodles average weight is 7 lbs")
    else:
        return("An average dog weights 50 lbs")

known_actions = {
    "calculate": calculate,
    "average_dog_weight": average_dog_weight
}

# def calculate(what):
#     return eval(what)

# def average_dog_weight(name):
#     weights = {
#         "Scottish Terrier": 20,
#         "Border Collie": 37,
#         "Toy Poodle": 7,
#     }
#     return weights.get(name, 50)

# known_actions = {
#     "calculate": calculate,
#     "average_dog_weight": average_dog_weight
# }

In [53]:
abot = Agent(prompt)

In [54]:
result = abot("How much does a toy poodle weigh?")
print(result)

Thought: I should look the dogs weight using average_dog_weight
Action: average_dog_weight: Toy Poodle
PAUSE


In [34]:
result = average_dog_weight("Toy Poodle")
result

7

In [35]:
next_prompt = "Observation: {}".format(result)
abot(next_prompt)

'Answer: A Toy Poodle weighs 7 lbs.'

In [37]:
abot.messages

[{'role': 'system',
  'content': 'You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\naverage_dog_weight:\ne.g. average_dog_weight: Collie\nreturns average weight of a dog when given the breed\n\nExample session:\n\nQuestion: How much does a Bulldog weigh?\nThought: I should look the dogs weight using average_dog_weight\nAction: average_dog_weight: Bulldog\nPAUSE\n\nYou will be called again with this:\n\nObservation: A Bulldog weights 51 lbs\n\nYou then output:\n\nAnswer: A bulldog weights 51 lbs'},
 {'role': 'user', 'content': 'How much does a 

In [38]:
abot = Agent(prompt)
question = """I have 2 dogs, a border collie and a scottish terrier. \
What is their combined weight"""
abot(question)

'Thought: I need to find the average weight of each breed and add them together\nAction: calculate: (average_dog_weight: Border Collie + average_dog_weight: Scottish Terrier)\nPAUSE'

In [40]:
next_prompt = "Observation: {}".format(average_dog_weight("Border Collie"))
print(next_prompt)

Observation: 37


In [41]:
abot(next_prompt)

'Answer: The combined weight of a Border Collie and a Scottish Terrier is 37.'

In [42]:
next_prompt = "Observation: {}".format(average_dog_weight("Scottish Terrier"))
print(next_prompt)

Observation: 20


In [43]:
abot(next_prompt)

'Thought: I need to find the average weight of each breed and add them together\nAction: calculate: (average_dog_weight: Border Collie + average_dog_weight: Scottish Terrier)\nPAUSE'

In [44]:
next_prompt = "Observation: {}".format(eval("37 + 20"))
print(next_prompt)

Observation: 57


In [45]:
abot(next_prompt)

'Answer: The combined weight of a Border Collie and a Scottish Terrier is 57.'

In [46]:
action_re = re.compile('^Action: (\w+): (.*)$')   # python regular expression to selection action

In [50]:
# def query(question, max_turns=5):
#     i = 0
#     bot = Agent(prompt)
#     next_prompt = question
#     while i < max_turns:
#         i += 1
#         result = bot(next_prompt)
#         print(result)
#         actions = [
#             action_re.match(a) 
#             for a in result.split('\n') 
#             if action_re.match(a)
#         ]
#         if actions:
#             # There is an action to run
#             action, action_input = actions[0].groups()
#             if action not in known_actions:
#                 raise Exception("Unknown action: {}: {}".format(action, action_input))
#             print(" -- running {} {}".format(action, action_input))
#             observation = known_actions[action](action_input)
#             print("Observation:", observation)
#             next_prompt = "Observation: {}".format(observation)
#         else:
#             return

def query(question, max_turns=5):
    i = 0
    bot = Agent(prompt)
    next_prompt = question
    while i < max_turns:
        i += 1
        result = bot(next_prompt)
        print(result)
        actions = [
            action_re.match(a) 
            for a in result.split('\n') 
            if action_re.match(a)
        ]
        if actions:
            # There is an action to run
            action, action_input = actions[0].groups()
            if action not in known_actions:
                raise Exception("Unknown action: {}: {}".format(action, action_input))
            print(" -- running {} {}".format(action, action_input))
            if action == "calculate":
                dog_weights = []
                for dog_name in action_input.split("+"):
                    dog_weight = known_actions["average_dog_weight"](dog_name.strip())
                    dog_weights.append(int(dog_weight.split(" ")[0]))
                observation = sum(dog_weights)
            else:
                observation = known_actions[action](action_input)
            print("Observation:", observation)
            next_prompt = "Observation: {}".format(observation)
        else:
            return

In [51]:
question = """I have 2 dogs, a border collie and a scottish terrier. \
What is their combined weight"""
query(question)

Thought: I need to find the average weight of each breed and add them together
Action: calculate: (average_dog_weight: Border Collie + average_dog_weight: Scottish Terrier)
PAUSE
 -- running calculate (average_dog_weight: Border Collie + average_dog_weight: Scottish Terrier)


ValueError: invalid literal for int() with base 10: 'An'